# python과 DAtabase 연동
1. 해당 라이브러리 필요: DB접속 드라이버,cx_Oracle, python-oracledb
    - 설치 : Anaconda prompt 창(관리자 권한) pip install 라이브러리이름
    - 주피터 노트북 설치: !pip install 설치모듈명
    - Anaconda : pip install : 파이썬에 호환이되는 or conda install : 파이썬 전용라이브러리는 아니지만  파이썬에서 쓸수는있는
    - pip install orcledb --upgrade : 최근 버전(권장)
    - pip install cx_Oracle --upgrade
    
2. 접속정보 확인
    - 사용자 계정과 비밀번호 : scott,1234
    - 오라클 url : localhost:1521/xe

# DB 연동 작업

In [1]:
# 라이브러리 임포트
import cx_Oracle as cx

## connection : 연결 통로 구축

In [2]:
conn=cx.connect('scott','1234','localhost:1521/xe')
# conn=cx.connect('scott/1234/localhost:1521/xe')

## db 테이블 조회

In [ ]:
# cursor() : 레코드셋을 탐색
##커서 객체 생성
cur=conn.cursor()
##쿼리 실행
sql='select * from product'
## 커서 객체를 이용해 sql 실행
cur.execute(sql)

for row in cur:
    print(row[0], row[1], row[2], end='')
    # clob 자료형은 그냥 읽어올수없다
    description=row[3].read()
    print(description, end='')
    print(row[4])

## 전체 레코드 삭제 후 테이블에 레코드 삽입하는 작업

In [ ]:
sql= 'delete from product'
cur.execute(sql)
conn.commit()

In [ ]:
# 파이썬에서 db에 레코드를 삽입
items=[(1, '레몬', 1500, '레몬이다', 'lemon.jpg'),
(2, '귤', 2500, '귤이다', 'mandarin.jpg'),
(3, '사과', 3500, '사과다', 'apple.jpg'),
(4, '포도', 4500,'포도다', 'grape.jpg'),
(5, '딸기', 5500, '딸기다', 'strawberry.jpg')      
]

for row in items:
    # : 필드의 인덱스 1부터 
    sql='insert into product values (:1,:2,:3,:4,:5)'
    print(row)
    cur.execute(sql,row)
conn.commit()

In [ ]:
# 상품개수를 추출하는 함수
sql= 'select count(*) from product'
cur.execute(sql)

# 레코드 개수가 1개일경우
count=cur.fetchone()
print('상품개수: ',count[0])

In [ ]:
# 레코드삭제
sql='delete from product'
cur.execute(sql)
conn.commit()

In [ ]:
# 레코드를 일괄적으로 추가
items=[{1, '레몬', 1500, '레몬이다', 'lemon.jpg'},
{2, '귤', 2500, '귤이다', 'mandarin.jpg'},
{3, '사과', 3500, '사과다', 'apple.jpg'},
{4, '포도', 4500,'포도다', 'grape.jpg'},
{5, '딸기', 5500, '딸기다', 'strawberry.jpg'}      
]
sql='insert into product values (:1,:2,:3,:4,:5)'
# 일괄적으로 레코드 추가할땐 레코드 개수 지정
cur.bindarraysize=len(items)
cur.executemany(sql,items)
conn.commit()

In [7]:
items = [
    {'idx': 40, 'pname': '레몬', '가격': 1500, 'description': '레몬이다', 'image': 'lemon.jpg'}
]

sql = 'insert into product values (:idx, :pname, :가격, :description, :image)'

# 커서 생성
cur = conn.cursor()

# 일괄적으로 레코드 추가할때 레코드 개수 지정
cur.bindarraysize = len(items)

# executemany 메서드에 딕셔너리의 리스트 전달
cur.executemany(sql, items)

# 변경 내용을 데이터베이스에 저장
conn.commit()

In [ ]:
cur.close()
conn.close()

# 회원관리 프로그램

- crud =DML
- 테이블 생성 Member(userid primary key ,userpwd,name)
- 주요기능 : 회원가입, 회원수정, 전체 회원정보 출력, 회원삭제

In [ ]:
import cx_Oracle as cx
conn=cx.connect('scott','1234','localhost:1521/xe')

## member 테이블 조회

In [ ]:
cur=conn.cursor()
sql='select * from Member'
cur.execute(sql)
# 커서 객체가 가진 모든 레코드를 resultset 객체에 담기 => 리스트 객체 반환
rs=cur.fetchall()
# print(rs)
for member in rs:
    print(member)

In [ ]:
def insert(userid,pwd,username):
    conn=cx.connect('scott','1234','localhost:1521/xe')
    cur=conn.cursor()
    sql='insert into Member values (:1,:2,:3)'
    cur.execute(sql,[userid,pwd,username])
    cur.close()
    conn.commit() # DB 반영
    conn.close()

## 레코드 삽입

In [ ]:
print('회원가입')
userid=input('id를 입력: ')
pwd=input('pwd를 입력: ')
username=input('이름입력: ')

insert(userid,pwd,username)
    

## 레코드수정

In [ ]:
def update(userid,pwd,username):
    conn=cx.connect('scott','1234','localhost:1521/xe')
    sql='update Member set pwd=:1, username=:2 where userid=:3'
    cur=conn.cursor()
    cur.execute(sql,[pwd,username,userid])
    cur.close()
    conn.commit()
    conn.close()
    

In [ ]:
# id_check(userid)
def id_check(userid):
    conn=cx.connect('scott','1234','localhost:1521/xe')
    SQL='select * from Member'
    cur=conn.cursor()
    cur.execute(sql)
    rs=cur.fetchall()
    
    for row in rs:
        if row[0]==userid:
            return True
    return False
    
    
    
    

In [ ]:
userid=input('id를 입력: ')
if id_check(userid):    
    pwd=input('pwd를 입력: ')
    username=input('이름입력: ')
    update(userid,pwd,username)
else: 
    print('회원정보없음')
    

In [ ]:
def delete(userid):
    conn=cx.connect('scott','1234','localhost:1521/xe')
    sql='delete from Member where userid=:1'
    cur=conn.cursor()
    cur.execute(sql,[userid])
    cur.close()
    conn.commit()
    conn.close()

In [ ]:
userid=input('id를 입력: ')
if id_check(userid):    
    delete(userid)
else: 
    print('회원정보없음')

# 최종 프로그램

## db연동

In [ ]:
import cx_Oracle as cx
conn=cx.connect('scott','1234','localhost:1521/xe')

## 기능별구현

In [ ]:
def insert(userid,pwd,username):
    conn=cx.connect('scott','1234','localhost:1521/xe')
    cur=conn.cursor()
    sql='insert into Member values (:1,:2,:3)'
    cur.execute(sql,[userid,pwd,username])
    cur.close()
    conn.commit()
    conn.close()
    
def update(userid,pwd,username):
    conn=cx.connect('scott','1234','localhost:1521/xe')
    sql='update Member set pwd=:1, username=:2 where userid=:3'
    cur=conn.cursor()
    cur.execute(sql,[pwd,username,userid])
    cur.close()
    conn.commit()
    conn.close()
    
def id_check(userid):
    conn=cx.connect('scott','1234','localhost:1521/xe')
    sql='select * from Member'
    cur=conn.cursor()
    cur.execute(sql)
    rs=cur.fetchall()
    
    for row in rs:
        if row[0]==userid:
            return True
    return False
    
def delete(userid):
    conn=cx.connect('scott','1234','localhost:1521/xe')
    sql='delete from Member where userid=:1'
    cur=conn.cursor()
    cur.execute(sql,userid)
    cur.close()
    conn.commit()
    conn.close()
    
def showall():
    cur=conn.cursor()
    sql='select * from Member'
    cur.execute(sql)
# 커서 객체가 가진 모든 레코드를 resultset 객체에 담기
    rs=cur.fetchall()
# print(rs)
    for member in rs:
        print(member)
    cur.close()
    conn.close()

## main

In [ ]:
while True:
    print('\n== 회원관리 플오글앰==')
    print('1.회원등록')
    print('2.회원수정')
    print('3. 회원 삭제')
    print('4. 전체회원')
    print('5. 종료')
    menu=int(input('메뉴 : '))
    if menu==1:
        userid=input('id를 입력: ')
        pwd=input('pwd를 입력: ')
        username=input('이름입력: ')
        insert(userid,pwd,username)

    elif menu==2:
        userid=input('id를 입력: ')
        if id_check(userid):
            pwd=input('pwd를 입력: ')
            username=input('이름입력: ')
            update(userid,pwd,username)
                    
    elif menu==3:
        userid=input('id를 입력: ')
        if id_check(userid):    
            delete(userid)
        else: 
            print('회원정보없음')
    elif menu==4:
        showall()
        
    elif menu==5:
        break
    else:
        print('다시입력')

In [ ]:
import dbTest
while True:
    print('\n== 회원관리 플오글앰==')
    print('1.회원등록')
    print('2.회원수정')
    print('3. 회원 삭제')
    print('4. 전체회원')
    print('5. 종료')
    menu=int(input('메뉴 : '))
    if menu==1:
        userid=input('id를 입력: ')
        pwd=input('pwd를 입력: ')
        username=input('이름입력: ')
        dbTest.insert(userid,pwd,username)

    elif menu==2:
        userid=input('id를 입력: ')
        if dbTest.check(userid) == 1:
            pwd=input('pwd를 입력: ')
            username=input('이름입력: ')
            dbTest.update(userid,pwd,username)
                    
    elif menu==3:
        userid=input('id를 입력: ')
        if dbTest.check(userid) == 1:    
            dbTest.delete(userid)
        else: 
            print('회원정보없음')
    elif menu==4:
        dbTest.listMember()
        
    elif menu==5:
        break
    else:
        print('다시입력')

In [ ]:
import dbTest